# Bibliotecas

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import funcoes_modelos as fmod
import funcoes_bow as fbow

import time

## Configurações

In [2]:
flag_cpu = True
if flag_cpu == True:
    # force CPU (make CPU visible)
    cpus = tf.config.experimental.list_physical_devices('CPU')
    print(cpus)
    tf.config.set_visible_devices([], 'GPU')  # hide the GPU
    tf.config.set_visible_devices(cpus[0], 'CPU') # unhide potentially hidden CPU
    tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Constantes

In [3]:
SEED = 42

# Leitura dos arquivos

In [4]:
arquivos = ['hn_balanceado', 'hn_desbalanceado', 'sn_balanceado', 'sn_desbalanceado']

lista_df = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/{arquivo}.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df.append(df)

In [5]:
#lista_df[0].dtypes

In [6]:
print(f"Tamanho dos Datasets:\n\n\t\t| Hard\t| Soft\t|\nBalanceado\t| {lista_df[0].shape[0]}\t| {lista_df[2].shape[0]}\t|\nDesbalanceado\t| {lista_df[1].shape[0]}\t| {lista_df[3].shape[0]}\t|")

Tamanho dos Datasets:

		| Hard	| Soft	|
Balanceado	| 8400	| 8400	|
Desbalanceado	| 13290	| 13290	|


# Separando o Dataset

In [7]:
def train_test_valid(df):
    
    X = df[["titulo_1", "titulo_2"]]
    y = df["match"].to_list()
    
    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED, stratify = y)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.285, random_state = SEED, stratify = y_train_valid)
    
    return X_train, y_train, X_test, y_test, X_valid, y_valid


In [8]:
lista_train_test_valid = []
for df in lista_df:
    lista_train_test_valid.append(train_test_valid(df))

In [9]:
for nome, dataset in zip(arquivos, lista_train_test_valid):    
    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

# Experimentos Com Os Modelos

## Funções

In [10]:
def salvar_df_resultado(report, modelo, nome_dataset, tempo_exec):
    
    df_resultado = pd.DataFrame(report).transpose()
    df_resultado['modelo'] = modelo
    df_resultado['dataset'] = nome_dataset
    df_resultado['tempo'] = tempo_exec

    df_resultado.to_csv(f'Dados/Resultados/{modelo}/{nome_dataset}_resultado.csv', index = True)
    
    return df_resultado

## BERT, roBERTa, XLMR e ELECTRA

In [11]:
a

NameError: name 'a' is not defined

In [ ]:
modelos = ("BERT", "roBERTa", "XLMR", "ELECTRA")

In [ ]:
for modelo in modelos:
    
    lista_df_resultado = []
    
    for nome, dataset in zip(arquivos, lista_train_test_valid):    
        
        X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

        start_time = time.time()
        (nome, historico, y_test, y_pred) = fmod.pipeline_bert(modelo, nome, X_train[:5], y_train[:5], X_valid[:5], y_valid[:5], X_test[:5], y_test[:5])
        end_time = time.time()
        runtime = end_time - start_time
        
        pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/{modelo}/{nome}_historico.csv', index = False)

        report = classification_report(y_test, y_pred, output_dict = True)
        
        df_resultado = salvar_df_resultado(report, modelo, nome, runtime)

        lista_df_resultado.append(df_resultado)

## BoW

In [12]:
fim = 5

In [13]:
import string
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
def remove_pontuacao(texto):
    
    #texto_sp = "".join([i for i in texto if i not in string.punctuation])
    texto_sp = texto.translate(str.maketrans('', '', string.punctuation))
    
    return texto_sp


#Tranforma o dataframe de dados em uma lista única de sentenças
def get_list_dataframe(dataframe):
    
    lista_titulos = []
    for titulo_1, titulo_2 in zip(dataframe["titulo_1"], dataframe["titulo_2"]):

        lista_titulos.append(remove_pontuacao(titulo_1))
        lista_titulos.append(remove_pontuacao(titulo_2))
        
    return lista_titulos


#Cria bow binário das sentenças
def vectorize_dataframe(dataframe):
    lista_titulos = get_list_dataframe(dataframe)

    vectorizer = CountVectorizer(analyzer = "word",
                                tokenizer = None,
                                lowercase = True,
                                strip_accents = 'unicode',
                                binary = True
                                ) 

    vector = vectorizer.fit_transform(lista_titulos).toarray()

    return vector, vectorizer


#Cria a matrix de coocorrencia do título1 e título2 
def get_cooccurrence_bow(dataframe):
    
    vector, vectorizer = vectorize_dataframe(dataframe)
    lista_features = []

    for indice in range(0, len(vector), 2):
        
        vec_titulo_1 = vector[indice]
        vec_titulo_2 = vector[indice + 1]
        
        lista_coo = np.multiply(np.logical_and(vec_titulo_1, vec_titulo_2), 1).tolist()
        
        lista_features.append(lista_coo)

    return lista_features, vector, vectorizer

In [43]:
modelo = "BoW"

for nome, dataset in zip(arquivos, lista_train_test_valid):    
    
    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

    #start_time = time.time()
    
    # X_train_vec, X_valid_vec, X_test_vec = fbow.pipeline_rf(nome, X_train, y_train, X_valid, y_valid, X_test, y_test)
    #name_dataset, y_test, label_pred = fbow.pipeline_rf(nome, X_train[:fim], y_train[:fim], X_valid[:fim], y_valid[:fim], X_test[:fim], y_test[:fim])
    co_treino, vec_treino, cv_treino = get_cooccurrence_bow(X_train)
    co_teste = get_cooccurrence_bow_test_val(cv_treino, cv_treino, X_test)
    
    #print(f"\n\n{nome}\n\n")

    break
    '''end_time = time.time()
    runtime = end_time - start_time

    report = classification_report(y_test, y_pred, output_dict = True)
    
    df_resultado = salvar_df_resultado(report, modelo, nome, runtime)

    lista_df_resultado.append(df_resultado)'''

In [42]:
def get_cooccurrence_bow_test_val(cv_treino, vectorizer_treino, X_):
    
    co_, vec_, cv_ = get_cooccurrence_bow(X_)
    
    co_vector = np.zeros((len(co_), len(vectorizer_treino.vocabulary_)), dtype = np.int64).tolist()
    
    for linha in range(len(co_)):

        for chave, dic_indice in zip(cv_.vocabulary_.keys(), cv_.vocabulary_.values()):
            
            if chave in cv_treino.vocabulary_:
                
                #print("linha:", linha)
                #print(co_[linha][dic_indice])
                co_vector[linha][dic_indice] = co_[linha][dic_indice]
                #print("dic_indice:", co_vector[dic_indice])

                             
    return co_vector

In [53]:
len(co_teste[3])

6722

In [ ]:
for i, (chave, dic_indice) in enumerate(zip(cv_treino.vocabulary_.keys(), cv_treino.vocabulary_.values())):
    
    print(i)
    print(chave)
    print(dic_indice)
    
    break

In [ ]:
vectorizer.vocabulary_['smartphone']

In [ ]:
vectorizer.vocabulary_

In [ ]:
X_train.head()

In [ ]:
len(X_train_vec[0])

In [ ]:
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

In [ ]:
df_resultado

In [ ]:
'''lista_df_resultado = []
for nome, df in zip(arquivos, lista_df):
    
    X = df[["titulo_1", "titulo_2"]]#[df["titulo_1"].to_list(), df["titulo_2"].to_list()]
    y = df["match"].to_list()

    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED, stratify = y)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.2, random_state = SEED, stratify = y_train_valid)

    (nome, historico, y_test, y_pred) = fmod.pipeline_bert(nome, X_train[:5], y_train[:5], X_valid[:5], y_valid[:5], X_test[:5], y_test[:5])

    pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/BERTo/{nome}_historico.csv', index = False)

    report = classification_report(y_test, y_pred, output_dict = True)
    df_resultado = pd.DataFrame(report).transpose()
    df_resultado['modelo'] = nome

    break

    #df_resultado.to_csv(f'Dados/Resultados/BERTo/{nome}_resultado.csv', index = True)

    lista_df_resultado.append(df_resultado)'''